In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df_train = pd.read_csv("./dataset/Train/train.csv").values
df_test = pd.read_csv("./dataset/Test/test.csv").values
df_sample = pd.read_csv("./dataset/Test/Sample_submission.csv").values

In [3]:
df_test.shape

(123, 1)

In [4]:
train_path = './dataset/Train/Images/'
test_path = './dataset/Test/Images/'
X = []
test = []
for each in df_train:
    img = cv2.imread(train_path + each[0])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (200, 200))
    X.append(img.flatten())
    
for each in df_test:
    img = cv2.imread(test_path + each[0])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (200, 200))
    test.append(img.flatten())


In [5]:
print(X[131].shape)
print(len(test))

(120000,)
123


In [6]:
Y = df_train[:, 1]
print(Y.shape)

(304,)


In [82]:
def distance(p1, p2):
    return np.sum((p2 - p1) ** 2) ** .5

def knn(X, Y, test, k = 4):
    m = len(X)
    d = {}
    for i in range(m):
        d[distance(test, X[i])] = Y[i]
    l = sorted(d)
    l = l[:k]
    label = [d.get(each) for each in l]
    label = np.unique(label, return_counts=True)
    idx = np.argmax(label[1])
    pred = label[0][idx]
    return pred

In [83]:
m = len(test)
outcome = np.empty((m, 2), dtype="<U10")
for i in range(m):
    ID = df_test[i][0]
    name = knn(X, Y, test[i])
    outcome[i][0] = ID
    outcome[i][1] = name
#     print(ID, name)

In [84]:
outcome[:8]

array([['1106.jpg', '0'],
       ['2236.jpg', '2'],
       ['1131.jpg', '1'],
       ['2221.jpg', '0'],
       ['2011.jpg', '0'],
       ['2030.jpg', '0'],
       ['2035.jpg', '0'],
       ['1110.jpg', '1']], dtype='<U10')

In [85]:
np.unique(outcome[:,1], return_counts=True)

(array(['0', '1', '2'], dtype='<U10'), array([75, 26, 22]))

In [86]:
pd.DataFrame(outcome, columns=["ImageId", "NameOfPokemon"]).to_csv("Outcome.csv", index=None)

In [235]:
r = pd.read_csv('Outcome.csv').values

In [52]:
print(type(r[0][0]))

<class 'str'>


### Prediction using Logistic Regression

In [13]:
X = np.asarray(X)
Y = np.asarray(Y)

In [14]:
print(X.shape)
print(Y.shape)

(304, 120000)
(304,)


In [87]:
# change the labels into number notation
Y = [0 if each == 'Pikachu' else 1 if each == 'Bulbasaur' else 2 for each in Y]

In [37]:
Y = np.reshape(Y, (-1, ))

In [38]:
print(Y.shape)

(304,)


In [39]:
from sklearn.linear_model import LogisticRegression

In [40]:
model = LogisticRegression(multi_class='ovr')

In [41]:
model.fit(X, Y)

/home/macab/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
test = np.asarray(test)

In [44]:
test.shape

(123, 120000)

In [64]:
predictions =  model.predict(test)

In [65]:
# get back the origin label
outcome = ['Pikachu' if each == 0 else 'Bulbasaur' if each == 1 else 'Charmander' for each in predictions]

In [66]:
outcome = np.asarray(outcome)

In [67]:
np.unique(outcome, return_counts=True)

(array(['Bulbasaur', 'Charmander', 'Pikachu'], dtype='<U10'),
 array([43, 41, 39]))

In [70]:
outcome = np.reshape(outcome, (-1, 1))

In [71]:
index = np.arange(outcome.shape[0])
index =  np.reshape(index, (-1, 1))

In [76]:
outcome = np.hstack((index, outcome))

In [78]:
pd.DataFrame(outcome, columns=["ImageId", "NameOfPokemon"]).to_csv("OutcomeLR.csv", index=None)

In [81]:
model.score?